# Generate training data

### CISCO and Spaninsh Ens

In [1]:
import pandas as pd
import re

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
CISCO_PATH = "Schemes/Cisco.csv"         # Assicurati che il percorso sia corretto
ENS_PATH = "Schemes/SpanishENS.csv"      # Assicurati che il percorso sia corretto
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step1_cisco_ens.csv"

# ==============================================================================
# 2. CARICAMENTO DATI
# ==============================================================================
print("1. Caricamento file...")

# Caricamento Cisco
df_cisco = pd.read_csv(CISCO_PATH)
print(f"   - Cisco loaded: {df_cisco.shape} righe")

# Caricamento Spanish ENS (con separatore punto e virgola)
# Usiamo dtype=str per evitare problemi con codici interpretati come numeri
df_ens = pd.read_csv(ENS_PATH, sep=';', dtype=str)
print(f"   - Spanish ENS loaded: {df_ens.shape} righe")

# ==============================================================================
# 3. CREAZIONE LOOKUP TABLE (ENS INGLESE AGGREGATO)
# ==============================================================================
print("\n2. Indicizzazione e Aggregazione ENS (English)...")

ens_lookup = {}

# Raggruppiamo per codice perché nel CSV lo stesso codice può apparire più volte
# (es. una riga per il titolo, una riga per le domande)
grouped_ens = df_ens.groupby('Codice')

for code, group in grouped_ens:
    # Normalizza la chiave (codice)
    if pd.isna(code): continue
    clean_code = str(code).strip().lower()
    
    # Raccogli tutti i testi INGLESI disponibili per questo codice
    texts = []
    
    for _, row in group.iterrows():
        # Titolo Inglese
        if 'Titolo_EN' in row and not pd.isna(row['Titolo_EN']):
            t = str(row['Titolo_EN']).strip()
            if t: texts.append(t)
            
        # Domande Inglese
        if 'Domande_EN' in row and not pd.isna(row['Domande_EN']):
            q = str(row['Domande_EN']).strip()
            # Rimuovi i pipe '|' che separano le domande e sostituisci con spazio
            q = q.replace('|', ' ')
            if q: texts.append(q)
            
    # Unisci tutto il testo raccolto per questo controllo
    full_text_en = ". ".join(texts)
    
    # Pulizia finale (spazi doppi, ecc)
    full_text_en = re.sub(r'\s+', ' ', full_text_en).strip()
    
    if full_text_en:
        ens_lookup[clean_code] = full_text_en

print(f"   - Indicizzati {len(ens_lookup)} codici ENS univoci con testo in Inglese.")

# ==============================================================================
# 4. MAPPING E GENERAZIONE DATASET
# ==============================================================================
print("\n3. Generazione coppie di training (Cisco -> ENS)...")

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# Colonne Cisco dove cercare i riferimenti ENS
ens_columns = ['Spanish ENS BASIC Control', 'Spanish ENS Medium Control', 'Spanish ENS High Control']

for idx, row in df_cisco.iterrows():
    # Anchor (Cisco)
    # Combiniamo Titolo e Wording per la massima ricchezza semantica
    if pd.isna(row['Control Reference']): continue
    
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    anchor_text = f"{title}: {wording}".strip()
    
    # Target (ENS)
    # Cerchiamo i codici nelle 3 colonne
    target_codes = []
    for col in ens_columns:
        if col in row and not pd.isna(row[col]):
            # Split per virgola in caso ci siano più controlli in una cella
            codes_in_cell = str(row[col]).split(',')
            target_codes.extend([c.strip().lower() for c in codes_in_cell])
    
    # Rimuovi duplicati e itera
    unique_targets = set(target_codes)
    
    for t_code in unique_targets:
        if not t_code: continue
        
        if t_code in ens_lookup:
            # MATCH TROVATO
            positive_text = ens_lookup[t_code]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_code,
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'Spanish ENS (EN)'
            })
            stats["matches"] += 1
        else:
            stats["missing"].add(t_code)

# ==============================================================================
# 5. SALVATAGGIO
# ==============================================================================
df_train = pd.DataFrame(training_data)
df_train.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO FINALE")
print("-" * 30)
print(f"Totale coppie generate: {len(df_train)}")
print(f"Codici mancanti (nel file ENS): {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:5]}")
print(f"\nFile salvato come: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_train[['anchor_text', 'target_text']].head(2))

1. Caricamento file...
   - Cisco loaded: (713, 32) righe
   - Spanish ENS loaded: (213, 8) righe

2. Indicizzazione e Aggregazione ENS (English)...
   - Indicizzati 169 codici ENS univoci con testo in Inglese.

3. Generazione coppie di training (Cisco -> ENS)...
------------------------------
RISULTATO FINALE
------------------------------
Totale coppie generate: 318
Codici mancanti (nel file ENS): 28
Esempio mancanti: ['op.cont.3', 'mp.per.9', 'mp.info.9', 'op.ext.9', 'op.exp.11']

File salvato come: TrainAndTestData/trainingData/training_data_step1_cisco_ens.csv

Anteprima dati:
                                         anchor_text  \
0  Control Self-Assessments: Independent Control ...   
1  Control Self-Assessments: Independent Control ...   

                                         target_text  
0  Metrics system. Taking into account the securi...  
1  Certified components. Is the Catalog of Inform...  


### CISCO and Spaninsh Ens

In [2]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
# Percorsi basati sulla tua struttura di cartelle
PATH_CISCO = "Schemes/Cisco.csv"
PATH_SECNUM = "Schemes/Secnumcloud.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step2_cisco_secNumCloud.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE SECNUMCLOUD
# ==============================================================================
print("1. Caricamento e indicizzazione SecNumCloud...")

try:
    # Tentativo di lettura standard (virgola)
    df_sec = pd.read_csv(PATH_SECNUM)
    
    # Verifica colonne critiche
    if 'ID' not in df_sec.columns or 'Description_EN' not in df_sec.columns:
        # Fallback: prova con punto e virgola se la prima lettura non ha parsato le colonne
        df_sec = pd.read_csv(PATH_SECNUM, sep=';')

    print(f"   - SecNumCloud caricato: {len(df_sec)} righe.")
except Exception as e:
    print(f"   - ERRORE critico nel caricamento di SecNumCloud: {e}")
    exit()

# Creazione dizionario di lookup: ID (lowercase) -> Descrizione Inglese
secnum_lookup = {}
for _, row in df_sec.iterrows():
    if pd.isna(row['ID']): continue
    
    # Normalizzazione ID (es: "5.1.A " -> "5.1.a")
    clean_id = str(row['ID']).strip().lower()
    
    # Estrazione testo inglese
    if 'Description_EN' in row and not pd.isna(row['Description_EN']):
        desc = str(row['Description_EN']).strip()
        secnum_lookup[clean_id] = desc

print(f"   - Indicizzati {len(secnum_lookup)} controlli SecNumCloud univoci (EN).")

# ==============================================================================
# 3. MAPPING CISCO -> SECNUMCLOUD
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> SecNumCloud)...")

df_cisco = pd.read_csv(PATH_CISCO)
print(f"   - Cisco caricato: {len(df_cisco)} righe.")

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# La colonna Cisco che contiene i riferimenti
target_col = 'SecNumCloud Control'

for idx, row in df_cisco.iterrows():
    # Verifica se c'è un mapping
    if target_col not in row or pd.isna(row[target_col]):
        continue
    
    # 1. Preparazione Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    # Uniamo titolo e wording per dare contesto completo al modello
    anchor_text = f"{title}: {wording}".strip()
    
    # 2. Preparazione Target (SecNumCloud IDs)
    # Cisco può contenere liste tipo "5.1.a, 5.2.b" o usare newline
    raw_refs = str(row[target_col]).replace('\n', ',')
    target_ids_list = [t.strip().lower() for t in raw_refs.split(',')]
    
    # 3. Matching
    for t_id in set(target_ids_list): # set() per evitare duplicati sulla stessa riga
        if not t_id: continue
        
        if t_id in secnum_lookup:
            # MATCH TROVATO
            positive_text = secnum_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,      # ID normalizzato
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'SecNumCloud (EN)'
            })
            stats["matches"] += 1
        else:
            # MATCH NON TROVATO (ID presente in Cisco ma non nel file SecNumCloud)
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 2
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 2")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici SecNumCloud non trovati: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:3]}...")
print(f"\nFile salvato: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione SecNumCloud...
   - SecNumCloud caricato: 287 righe.
   - Indicizzati 261 controlli SecNumCloud univoci (EN).

2. Generazione coppie di training (Cisco -> SecNumCloud)...
   - Cisco caricato: 713 righe.
------------------------------
RISULTATO STEP 2
------------------------------
Nuove coppie generate: 567
Codici SecNumCloud non trovati: 33
Esempio mancanti: ['12.7.d', '8.5.b', '5.3.e']...

File salvato: TrainAndTestData/trainingData/training_data_step2_cisco_secNumCloud.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1  18.2.1.a  The service provider must document and impleme...
1     CCF 1    18.4.a  The service provider must document and impleme...


### Cisco and BSIC5

In [3]:
import pandas as pd
import json
import os
import re

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_CISCO = "Schemes/Cisco.csv"
PATH_BSI = "Schemes/BSI-C5.json"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step3_cisco_bsi.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE BSI C5 (JSON)
# ==============================================================================
print("1. Caricamento e indicizzazione BSI C5...")

try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
    print(f"   - BSI C5 JSON caricato. Totale elementi: {len(bsi_data)}")
except Exception as e:
    print(f"   - ERRORE CRITICO nel caricamento del JSON BSI: {e}")
    # In un notebook, fermiamo l'esecuzione qui se il file non c'è
    raise e

bsi_lookup = {}

for item in bsi_data:
    # Verifica che il codice esista
    if 'code' not in item or not item['code']:
        continue
        
    # Normalizzazione ID (es. "IAM-02" -> "iam-02")
    code = str(item['code']).strip().lower()
    
    # Costruzione del Testo Ricco
    # Combiniamo Nome, Titolo Descrizione e Descrizione Estesa
    name = item.get('name', '')
    desc_title = item.get('descriptionTitle', '')
    desc_ext = item.get('descriptionExtended', '')
    
    # Pulizia valori None/Null (che nel JSON appaiono come null)
    if name is None: name = ""
    if desc_title is None: desc_title = ""
    if desc_ext is None: desc_ext = ""
    
    # Unione stringhe
    full_text = f"{name}: {desc_title} {desc_ext}".strip()
    
    # Pulizia spazi extra e newline
    full_text = re.sub(r'\s+', ' ', full_text).strip()
    
    if full_text:
        bsi_lookup[code] = full_text

print(f"   - Indicizzati {len(bsi_lookup)} controlli BSI con testo.")

# ==============================================================================
# 3. MAPPING CISCO -> BSI C5
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> BSI C5)...")

try:
    df_cisco = pd.read_csv(PATH_CISCO)
    print(f"   - Cisco CSV caricato: {len(df_cisco)} righe.")
except Exception as e:
    print(f"   - ERRORE CRITICO nel caricamento Cisco: {e}")
    raise e

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# La colonna in Cisco si chiama esattamente "BSI C5"
target_col = 'BSI C5'

for idx, row in df_cisco.iterrows():
    # Verifica se c'è un mapping in questa riga
    if target_col not in row or pd.isna(row[target_col]):
        continue
        
    # 1. Preparazione Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    anchor_text = f"{title}: {wording}".strip()
    anchor_text = re.sub(r'\s+', ' ', anchor_text) # pulizia veloce spazi
    
    # 2. Preparazione Target (BSI IDs)
    # I codici in Cisco sono separati da virgola (es. "OIS-01, SP-01")
    raw_refs = str(row[target_col]).replace('\n', ',')
    target_ids_list = [t.strip().lower() for t in raw_refs.split(',')]
    
    # 3. Matching
    for t_id in set(target_ids_list): # set() per rimuovere duplicati
        if not t_id: continue
        
        if t_id in bsi_lookup:
            # MATCH TROVATO
            positive_text = bsi_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,      # ID normalizzato
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'BSI C5'
            })
            stats["matches"] += 1
        else:
            # MATCH NON TROVATO
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 3
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 3")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici BSI citati in Cisco ma non trovati nel JSON: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempi codici mancanti: {list(stats['missing'])[:5]}...")
print(f"\nFile salvato in: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione BSI C5...
   - BSI C5 JSON caricato. Totale elementi: 294
   - Indicizzati 223 controlli BSI con testo.

2. Generazione coppie di training (Cisco -> BSI C5)...
   - Cisco CSV caricato: 713 righe.
------------------------------
RISULTATO STEP 3
------------------------------
Nuove coppie generate: 555
Codici BSI citati in Cisco ma non trovati nel JSON: 0

File salvato in: TrainAndTestData/trainingData/training_data_step3_cisco_bsi.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1     sp-01  Documentation, communication and provision of ...
1     CCF 1     sp-02  Review and Approval of Policies and Instructio...


### Cisco and NewEUCS Requirements

In [4]:
import pandas as pd
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_CISCO = "Schemes/Cisco.csv"
PATH_EUCS = "Schemes/NewEucsRequirements_with_texts.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step4_cisco_eucs.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE EUCS
# ==============================================================================
print("1. Caricamento e indicizzazione EUCS...")

try:
    # Tentativo primario con virgola (basato sul tuo 'head')
    df_eucs = pd.read_csv(PATH_EUCS, sep=',', encoding='utf-8')
    
    # Verifica se le colonne chiave esistono, altrimenti riprova con ;
    if 'EUCS Ref (Detailed)' not in df_eucs.columns:
        print("   - Separatore virgola fallito, provo punto e virgola...")
        df_eucs = pd.read_csv(PATH_EUCS, sep=';', encoding='utf-8')
        
    print(f"   - EUCS caricato: {len(df_eucs)} righe.")
except Exception as e:
    print(f"   - ERRORE CRITICO caricamento EUCS: {e}")
    # Fallback encoding latin-1 se utf-8 fallisce
    try:
        df_eucs = pd.read_csv(PATH_EUCS, sep=',', encoding='latin-1')
        print("   - EUCS caricato con encoding latin-1.")
    except:
        raise e

eucs_lookup = {}

for _, row in df_eucs.iterrows():
    # La chiave è il riferimento dettagliato (es. OIS-01.1)
    if 'EUCS Ref (Detailed)' not in row or pd.isna(row['EUCS Ref (Detailed)']):
        continue
        
    # Normalizzazione ID
    code = str(row['EUCS Ref (Detailed)']).strip().lower()
    
    # Estrazione testo
    text = ""
    if 'EUCS Text' in row and not pd.isna(row['EUCS Text']):
        text = str(row['EUCS Text']).strip()
        # Pulizia caratteri strani (es. newline o pipe)
        text = text.replace('\n', ' ').replace('|', ' ')
        text = re.sub(r'\s+', ' ', text)
        
    if code and text:
        eucs_lookup[code] = text

print(f"   - Indicizzati {len(eucs_lookup)} requisiti EUCS con testo.")

# ==============================================================================
# 3. MAPPING CISCO -> EUCS
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> EUCS)...")

df_cisco = pd.read_csv(PATH_CISCO)
training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# Colonne target in Cisco (ne abbiamo 3 per l'EUCS)
target_columns = [
    'EUCS Basic Control', 
    'EUCS Substantial Control', 
    'EUCS High Control'
]

for idx, row in df_cisco.iterrows():
    # 1. Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    anchor_text = f"{title}: {wording}".strip()
    anchor_text = re.sub(r'\s+', ' ', anchor_text)
    
    # 2. Raccolta Target IDs da tutte e 3 le colonne
    target_ids_list = []
    for col in target_columns:
        if col in row and not pd.isna(row[col]):
            # Split per virgola o newline
            raw_val = str(row[col]).replace('\n', ',')
            codes = [c.strip().lower() for c in raw_val.split(',')]
            target_ids_list.extend(codes)
            
    # 3. Matching
    for t_id in set(target_ids_list): # set() rimuove duplicati tra le colonne
        if not t_id: continue
        
        # Correzione typo comune (visto nel file Cisco: OSI -> OIS)
        if t_id.startswith('osi-'):
            t_id = t_id.replace('osi-', 'ois-')

        if t_id in eucs_lookup:
            # MATCH TROVATO
            positive_text = eucs_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'EUCS'
            })
            stats["matches"] += 1
        else:
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 4
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 4")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici EUCS non trovati: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:5]}...")
print(f"\nFile salvato in: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione EUCS...
   - EUCS caricato: 529 righe.
   - Indicizzati 522 requisiti EUCS con testo.

2. Generazione coppie di training (Cisco -> EUCS)...
------------------------------
RISULTATO STEP 4
------------------------------
Nuove coppie generate: 1272
Codici EUCS non trovati: 14
Esempio mancanti: ['cs-09.2', 'ps-04.4a', 'cs-03.2a', 'doc-06.3', 'ccm-02.5']...

File salvato in: TrainAndTestData/trainingData/training_data_step4_cisco_eucs.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1  isp-02.5  In case of a delegation, the authorized bodies...
1     CCF 1  ois-02.3  The CSP shall implement the mitigating measure...


### NewEucs and Fabasoft

In [5]:
import pandas as pd
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_EUCS = "Schemes/NewEucsRequirements_with_texts.csv"
PATH_FABASOFT = "Schemes/fabasoftMetrics.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step5_fabasoft_eucs.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE EUCS (VIA BSI C5)
# ==============================================================================
print("1. Caricamento EUCS e indicizzazione via BSI C5...")

df_eucs = pd.DataFrame()

# Lettura file con pulizia colonne
try:
    # Usiamo engine python per gestire CSV complessi
    df_eucs = pd.read_csv(PATH_EUCS, sep=',', encoding='utf-8', engine='python', on_bad_lines='skip')
    
    # --- FIX CRUCIALE: RIMUOVIAMO SPAZI DAI NOMI DELLE COLONNE ---
    df_eucs.columns = df_eucs.columns.str.strip()
    
    print(f"   - EUCS caricato: {len(df_eucs)} righe.")
    print(f"   - Colonne trovate: {df_eucs.columns.tolist()}") # Debug
except Exception as e:
    print(f"   - Errore caricamento EUCS (utf-8): {e}")
    # Fallback latin-1 e separatore ;
    try:
        df_eucs = pd.read_csv(PATH_EUCS, sep=';', encoding='latin-1', engine='python', on_bad_lines='skip')
        df_eucs.columns = df_eucs.columns.str.strip()
        print(f"   - EUCS caricato (latin-1, ;): {len(df_eucs)} righe.")
    except Exception as e2:
        print(f"   - ERRORE CRITICO FILE EUCS: {e2}")

# Individuazione colonne target dopo lo strip
col_bsi = 'C5.2020 GERMANY'
col_eucs_id = 'EUCS Ref (Detailed)'
col_eucs_text = 'EUCS Text'

# Verifica esistenza colonna BSI
if col_bsi not in df_eucs.columns:
    print(f"   - ATTENZIONE: Colonna '{col_bsi}' non trovata anche dopo la pulizia!")
    # Tentativo di trovarla parzialmente
    candidates = [c for c in df_eucs.columns if "C5" in c and "GERMANY" in c]
    if candidates:
        col_bsi = candidates[0]
        print(f"   - Usando colonna alternativa: '{col_bsi}'")

bsi_to_eucs = {}
count_mapped = 0

if not df_eucs.empty and col_bsi in df_eucs.columns:
    for _, row in df_eucs.iterrows():
        # Estrazione dati EUCS
        e_id = str(row[col_eucs_id]).strip() if col_eucs_id in row and not pd.isna(row[col_eucs_id]) else ""
        e_text = str(row[col_eucs_text]).strip() if col_eucs_text in row and not pd.isna(row[col_eucs_text]) else ""
        
        if not e_id or not e_text: 
            continue
            
        # Estrazione dati BSI (Link)
        bsi_refs = str(row[col_bsi]) if not pd.isna(row[col_bsi]) else ""
        if not bsi_refs: 
            continue
            
        # Split per newline o virgola (es. "OIS-01\nOIS-03")
        refs = re.split(r'[\n,]', bsi_refs)
        
        for r in refs:
            clean_bsi = r.strip().lower()
            if clean_bsi:
                if clean_bsi not in bsi_to_eucs:
                    bsi_to_eucs[clean_bsi] = []
                bsi_to_eucs[clean_bsi].append((e_id, e_text))
                count_mapped += 1

print(f"   - Indicizzati {len(bsi_to_eucs)} codici BSI che puntano a {count_mapped} requisiti EUCS.")

# ==============================================================================
# 3. ELABORAZIONE FABASOFT METRICS
# ==============================================================================
print("\n2. Elaborazione Fabasoft Metrics -> EUCS (via BSI)...")

try:
    df_fab = pd.read_csv(PATH_FABASOFT, engine='python', on_bad_lines='skip')
    print(f"   - Fabasoft caricato: {len(df_fab)} righe.")
except Exception as e:
    print(f"   - ERRORE Fabasoft: {e}")
    df_fab = pd.DataFrame()

# Trova colonna riferimenti (contiene "Possible Control ID")
ref_col = None
for col in df_fab.columns:
    if "Possible Control ID" in col:
        ref_col = col
        break
if not ref_col and not df_fab.empty:
    ref_col = df_fab.columns[-1] # Fallback

print(f"   - Colonna Riferimenti Fabasoft: '{ref_col}'")

training_data = []
stats = {"matches": 0, "missing_bsi_link": set()}

if not df_fab.empty and ref_col and bsi_to_eucs:
    for idx, row in df_fab.iterrows():
        # Anchor (Metric)
        m_id = str(row.get('ID', ''))
        m_name = str(row.get('Name', ''))
        m_desc = str(row.get('Description', ''))
        
        # Pulizia nan
        if m_id.lower() == 'nan': m_id = ""
        if m_name.lower() == 'nan': m_name = ""
        if m_desc.lower() == 'nan': m_desc = ""
        
        anchor_parts = [p for p in [m_id, m_name, m_desc] if p]
        anchor_text = ": ".join(anchor_parts)
        anchor_text = re.sub(r'\s+', ' ', anchor_text).strip()
        
        if not anchor_text: continue
        
        # Estrazione Referenze BSI da Fabasoft
        raw_refs = str(row[ref_col]) if not pd.isna(row[ref_col]) else ""
        lines = raw_refs.split('\n')
        
        for line in lines:
            line = line.strip()
            target_bsi = None
            
            # Cerca "BSI C5 OPS-22"
            if "BSI C5" in line:
                target_bsi = re.sub(r'BSI\s+C5\s+', '', line, flags=re.IGNORECASE).strip()
            
            if target_bsi:
                clean_bsi = target_bsi.lower()
                
                # BRIDGE: Fabasoft(BSI) -> EUCS(BSI)
                if clean_bsi in bsi_to_eucs:
                    eucs_targets = bsi_to_eucs[clean_bsi]
                    for (e_id, e_text) in eucs_targets:
                        training_data.append({
                            'anchor_id': m_id,
                            'anchor_text': anchor_text,
                            'target_id': e_id,
                            'target_text': e_text,
                            'source': 'Fabasoft Metrics',
                            'target': 'EUCS (via BSI C5)'
                        })
                        stats["matches"] += 1
                else:
                    stats["missing_bsi_link"].add(clean_bsi)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
df_result = pd.DataFrame(training_data, columns=['anchor_id', 'anchor_text', 'target_id', 'target_text', 'source', 'target'])
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 5")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
if stats['missing_bsi_link']:
    print(f"Codici BSI in Fabasoft non mappati in EUCS: {len(stats['missing_bsi_link'])}")
print(f"\nFile salvato in: {OUTPUT_FILE}")

if not df_result.empty:
    print(df_result[['anchor_id', 'target_id']].head(3))

1. Caricamento EUCS e indicizzazione via BSI C5...
   - EUCS caricato: 529 righe.
   - Colonne trovate: ['EUCS Category', 'EUCS Control (2022)', 'Control_Code', 'EUCS Ref (Detailed)', 'EUCS Text', 'EUCS Ass. Level', 'Code', 'C5.2020 GERMANY', 'SecNumCloud FRANCE', 'ISO 27002', 'ISO 27017']
   - Indicizzati 110 codici BSI che puntano a 576 requisiti EUCS.

2. Elaborazione Fabasoft Metrics -> EUCS (via BSI)...
   - Fabasoft caricato: 57 righe.
   - Colonna Riferimenti Fabasoft: 'Possible Control ID
Scheme'
------------------------------
RISULTATO STEP 5
------------------------------
Nuove coppie generate: 206
Codici BSI in Fabasoft non mappati in EUCS: 1

File salvato in: TrainAndTestData/trainingData/training_data_step5_fabasoft_eucs.csv
                         anchor_id target_id
0  NumberOfKnownLowVulnerabilities  OPS-17.1
1  NumberOfKnownLowVulnerabilities  OPS-17.2
2  NumberOfKnownLowVulnerabilities  OPS-17.3


### BSIC5 and Fabasoft

In [6]:
import pandas as pd
import json
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_FABASOFT = "Schemes/fabasoftMetrics.csv"
PATH_BSI = "Schemes/BSI-C5.json"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step6_fabasoft_bsi.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE BSI C5 (JSON)
# ==============================================================================
print("1. Caricamento e indicizzazione BSI C5...")

try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
    print(f"   - BSI C5 JSON caricato. Totale elementi: {len(bsi_data)}")
except Exception as e:
    print(f"   - ERRORE CRITICO BSI: {e}")
    exit()

bsi_lookup = {}

for item in bsi_data:
    if 'code' not in item or not item['code']:
        continue
        
    # Normalizzazione ID (es. "OPS-22" -> "ops-22")
    code = str(item['code']).strip().lower()
    
    # Costruzione Testo Ricco
    name = item.get('name', '') or ""
    desc_title = item.get('descriptionTitle', '') or ""
    desc_ext = item.get('descriptionExtended', '') or ""
    
    full_text = f"{name}: {desc_title} {desc_ext}".strip()
    full_text = re.sub(r'\s+', ' ', full_text).strip()
    
    if full_text:
        bsi_lookup[code] = full_text

print(f"   - Indicizzati {len(bsi_lookup)} controlli BSI con testo.")

# ==============================================================================
# 3. ELABORAZIONE FABASOFT METRICS -> BSI DIRECT
# ==============================================================================
print("\n2. Elaborazione Fabasoft Metrics -> BSI C5 (Direct)...")

try:
    # Engine python per gestire header multi-riga e quote
    df_fab = pd.read_csv(PATH_FABASOFT, engine='python', on_bad_lines='skip')
    print(f"   - Fabasoft caricato: {len(df_fab)} righe.")
except Exception as e:
    print(f"   - ERRORE Fabasoft: {e}")
    exit()

# Trova la colonna giusta (quella che contiene "Possible Control ID")
ref_col = None
for col in df_fab.columns:
    if "Possible Control ID" in col:
        ref_col = col
        break
# Fallback
if not ref_col and not df_fab.empty:
    ref_col = df_fab.columns[-1]

print(f"   - Colonna Riferimenti Fabasoft: '{ref_col}'")

training_data = []
stats = {"matches": 0, "missing": set()}

if not df_fab.empty and ref_col:
    for idx, row in df_fab.iterrows():
        # 1. Anchor (Metrica)
        m_id = str(row.get('ID', ''))
        m_name = str(row.get('Name', ''))
        m_desc = str(row.get('Description', ''))
        
        # Pulizia stringhe "nan"
        if m_id.lower() == 'nan': m_id = ""
        if m_name.lower() == 'nan': m_name = ""
        if m_desc.lower() == 'nan': m_desc = ""
        
        anchor_parts = [p for p in [m_id, m_name, m_desc] if p]
        anchor_text = ": ".join(anchor_parts)
        anchor_text = re.sub(r'\s+', ' ', anchor_text).strip()
        
        if not anchor_text: continue
        
        # 2. Estrazione Riferimenti BSI
        raw_refs = str(row[ref_col]) if not pd.isna(row[ref_col]) else ""
        
        # Il contenuto è tipo "BSI C5 OPS-22\nBSI C5 PSS-02"
        lines = raw_refs.split('\n')
        
        for line in lines:
            line = line.strip()
            
            # Regex per estrarre il codice dopo "BSI C5"
            # Cerca "BSI C5" (case insensitive) seguito da spazi opzionali e poi il codice
            match = re.search(r'BSI\s*C5\s*([\w-]+)', line, re.IGNORECASE)
            
            if match:
                target_code = match.group(1).lower() # es. "ops-22"
                
                # 3. Matching
                if target_code in bsi_lookup:
                    positive_text = bsi_lookup[target_code]
                    
                    training_data.append({
                        'anchor_id': m_id,
                        'anchor_text': anchor_text,
                        'target_id': target_code,
                        'target_text': positive_text,
                        'source': 'Fabasoft Metrics',
                        'target': 'BSI C5'
                    })
                    stats["matches"] += 1
                else:
                    stats["missing"].add(target_code)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 6")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
if stats['missing']:
    print(f"Codici BSI mancanti: {len(stats['missing'])} (es: {list(stats['missing'])[:3]})")
print(f"\nFile salvato in: {OUTPUT_FILE}")

if not df_result.empty:
    print(df_result[['anchor_id', 'target_id']].head(3))

1. Caricamento e indicizzazione BSI C5...
   - BSI C5 JSON caricato. Totale elementi: 294
   - Indicizzati 223 controlli BSI con testo.

2. Elaborazione Fabasoft Metrics -> BSI C5 (Direct)...
   - Fabasoft caricato: 57 righe.
   - Colonna Riferimenti Fabasoft: 'Possible Control ID
Scheme'
------------------------------
RISULTATO STEP 6
------------------------------
Nuove coppie generate: 27
Codici BSI mancanti: 1 (es: ['idm-11'])

File salvato in: TrainAndTestData/trainingData/training_data_step6_fabasoft_bsi.csv
                            anchor_id target_id
0     NumberOfKnownLowVulnerabilities    ops-22
1     NumberOfKnownLowVulnerabilities    pss-02
2  NumberOfKnownMediumVulnerabilities    ops-22


### newEUCS and BSIC5 no old data overlap from oldEUCS

In [7]:
import pandas as pd
import json
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_NEW_EUCS = "Schemes/NewEucsRequirements_with_texts.csv"
PATH_OLD_EUCS = "Schemes/OldEucsRequirements.csv"
PATH_BSI = "Schemes/BSI-C5.json"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step7_neweucs_bsi_filtered.csv"

# ==============================================================================
# 2. CARICAMENTO "OLD EUCS" (DA ESCLUDERE)
# ==============================================================================
print("1. Caricamento Old EUCS (Blacklist)...")
old_eucs_codes = set()

try:
    # Provo a leggere il vecchio file
    df_old = pd.read_csv(PATH_OLD_EUCS)
    
    # Identifico la colonna ID (dal tuo head sembra 'controlId')
    col_id_old = None
    for c in df_old.columns:
        if 'controlId' in c or 'ID' in c:
            col_id_old = c
            break
            
    if col_id_old:
        # Normalizzo e salvo nel set
        old_eucs_codes = set(df_old[col_id_old].dropna().astype(str).str.strip().str.lower())
        print(f"   - Trovati {len(old_eucs_codes)} codici 'Old EUCS' da escludere dal training.")
    else:
        print("   - ATTENZIONE: Colonna ID non trovata in Old EUCS. Nessun filtro applicato!")

except Exception as e:
    print(f"   - Errore caricamento Old EUCS: {e}")
    print("   - Procedo senza filtrare (rischio data leakage se questo file serviva per il test).")

# ==============================================================================
# 3. INDICIZZAZIONE BSI C5 (TARGET)
# ==============================================================================
print("\n2. Indicizzazione BSI C5...")
bsi_lookup = {}

try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
        
    for item in bsi_data:
        if 'code' not in item: continue
        
        # ID BSI normalizzato
        code = str(item['code']).strip().lower()
        
        # Testo Ricco
        name = item.get('name', '') or ""
        desc_t = item.get('descriptionTitle', '') or ""
        desc_e = item.get('descriptionExtended', '') or ""
        
        full_text = f"{name}: {desc_t} {desc_e}".strip()
        full_text = re.sub(r'\s+', ' ', full_text).strip()
        
        if full_text:
            bsi_lookup[code] = full_text

    print(f"   - Indicizzati {len(bsi_lookup)} controlli BSI.")

except Exception as e:
    print(f"   - ERRORE CRITICO BSI: {e}")
    exit()

# ==============================================================================
# 4. MAPPING NEW_EUCS -> BSI (CON FILTRO)
# ==============================================================================
print("\n3. Generazione coppie NewEUCS -> BSI (Filtrate)...")

training_data = []
stats = {
    "total_candidates": 0,
    "excluded_by_filter": 0,
    "matches": 0,
    "missing_bsi": 0
}

try:
    # Caricamento New EUCS con pulizia colonne
    df_new = pd.read_csv(PATH_NEW_EUCS, sep=',', engine='python', on_bad_lines='skip')
    # Se fallisce virgola, prova ;
    if 'C5.2020 GERMANY' not in df_new.columns and len(df_new.columns) < 5:
         df_new = pd.read_csv(PATH_NEW_EUCS, sep=';', engine='python', on_bad_lines='skip')
         
    df_new.columns = df_new.columns.str.strip() # Rimuove spazi dai nomi colonne

    # Colonne chiave
    col_new_id = 'EUCS Ref (Detailed)' # ID univoco del controllo EUCS
    col_new_text = 'EUCS Text'
    col_bsi_ref = 'C5.2020 GERMANY'

    # Verifica colonne
    if col_new_id not in df_new.columns or col_bsi_ref not in df_new.columns:
        print(f"   - Colonne mancanti in New EUCS. Trovate: {df_new.columns.tolist()}")
        exit()

    for idx, row in df_new.iterrows():
        # 1. Analisi Anchor (EUCS)
        eucs_id_raw = str(row[col_new_id])
        eucs_id_clean = eucs_id_raw.strip().lower()
        
        eucs_text = str(row[col_new_text]) if col_new_text in row else ""
        eucs_text = re.sub(r'\s+', ' ', eucs_text).strip()

        if pd.isna(row[col_new_id]) or not eucs_text:
            continue

        # 2. FILTRO ANTI-LEAKAGE
        # Se questo ID EUCS è presente nel vecchio set (OldEucs), SALTALO.
        if eucs_id_clean in old_eucs_codes:
            stats["excluded_by_filter"] += 1
            continue

        # 3. Analisi Target (BSI)
        bsi_refs_raw = str(row[col_bsi_ref])
        if pd.isna(row[col_bsi_ref]) or not bsi_refs_raw.strip():
            continue

        # Split refs (newline o virgola)
        refs = re.split(r'[\n,]', bsi_refs_raw)
        
        for r in refs:
            stats["total_candidates"] += 1
            target_bsi = r.strip().lower()
            
            if not target_bsi: continue

            if target_bsi in bsi_lookup:
                # MATCH!
                target_text = bsi_lookup[target_bsi]
                
                training_data.append({
                    'anchor_id': eucs_id_raw, # Mantengo ID originale per leggibilità
                    'anchor_text': eucs_text,
                    'target_id': target_bsi,
                    'target_text': target_text,
                    'source': 'NewEUCS (Filtered)',
                    'target': 'BSI C5'
                })
                stats["matches"] += 1
            else:
                stats["missing_bsi"] += 1

except Exception as e:
    print(f"   - ERRORE durante il processing: {e}")

# ==============================================================================
# 5. SALVATAGGIO
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 7 (FILTRATO)")
print("-" * 30)
print(f"Controlli Old EUCS trovati (Blacklist): {len(old_eucs_codes)}")
print(f"Coppie potenziali scartate perché nel Test Set: {stats['excluded_by_filter']}")
print(f"Nuove coppie valide generate: {len(df_result)}")
print(f"File salvato in: {OUTPUT_FILE}")

if not df_result.empty:
    print("\nAnteprima:")
    print(df_result[['anchor_id', 'target_id']].head(3))

1. Caricamento Old EUCS (Blacklist)...
   - Trovati 70 codici 'Old EUCS' da escludere dal training.

2. Indicizzazione BSI C5...
   - Indicizzati 223 controlli BSI.

3. Generazione coppie NewEUCS -> BSI (Filtrate)...
------------------------------
RISULTATO STEP 7 (FILTRATO)
------------------------------
Controlli Old EUCS trovati (Blacklist): 70
Coppie potenziali scartate perché nel Test Set: 0
Nuove coppie valide generate: 569
File salvato in: TrainAndTestData/trainingData/training_data_step7_neweucs_bsi_filtered.csv

Anteprima:
  anchor_id target_id
0  OIS-01.1    ois-01
1  OIS-01.2    ois-01
2  OIS-01.3    ois-01


## Merge to creare train data

In [8]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
DATA_DIR = "TrainAndTestData/trainingData/"
OUTPUT_MASTER = "TrainAndTestData/trainingData/MASTER_TRAINING_DATA.csv"

# Lista completa dei 7 file (Tutti gli step fatti finora)
FILES_TO_MERGE = [
    "training_data_step1_cisco_ens.csv",            # Cisco -> Spanish ENS
    "training_data_step2_cisco_secNumCloud.csv",    # Cisco -> SecNumCloud
    "training_data_step3_cisco_bsi.csv",            # Cisco -> BSI C5
    "training_data_step4_cisco_eucs.csv",           # Cisco -> EUCS (Originale)
    "training_data_step5_fabasoft_eucs.csv",        # Fabasoft -> EUCS
    "training_data_step6_fabasoft_bsi.csv",         # Fabasoft -> BSI C5
    "training_data_step7_neweucs_bsi_filtered.csv"  # NewEUCS -> BSI (Filtrato Old)
]

# ==============================================================================
# 2. UNIONE (MERGE)
# ==============================================================================
print("Inizio creazione del MASTER DATASET COMPLETO...")

df_list = []

for filename in FILES_TO_MERGE:
    file_path = os.path.join(DATA_DIR, filename)
    
    if os.path.exists(file_path):
        print(f"   - Lettura: {filename} ...", end="")
        try:
            df = pd.read_csv(file_path)
            # Aggiungiamo colonna origine per tracciabilità
            df['original_file'] = filename
            df_list.append(df)
            print(f" OK ({len(df)} righe)")
        except Exception as e:
            print(f" ERRORE: {e}")
    else:
        print(f"   - ATTENZIONE: File mancante: {filename}")

if not df_list:
    print("ERRORE: Nessun file trovato!")
    exit()

# Concatenazione
master_df = pd.concat(df_list, ignore_index=True)

# ==============================================================================
# 3. SHUFFLE E PULIZIA
# ==============================================================================
print("\nMescolamento e Pulizia Finale...")

# 1. Shuffle (Mescolamento casuale)
# shuffle al 100% per rompere l'ordine dei file
master_df = master_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 2. Rimozione duplicati esatti
# Se una coppia è identica (stessa ancora, stesso target), la teniamo una volta sola
initial_len = len(master_df)
master_df.drop_duplicates(subset=['anchor_text', 'target_text'], inplace=True)
dedup_len = len(master_df)

if initial_len != dedup_len:
    print(f"   - Rimossi {initial_len - dedup_len} duplicati esatti.")

# 3. Pulizia NaN
master_df.dropna(subset=['anchor_text', 'target_text'], inplace=True)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
master_df.to_csv(OUTPUT_MASTER, index=False, encoding='utf-8')

print("-" * 30)
print("RISULTATO FINALE (MASTER DATASET)")
print("-" * 30)
print(f"Totale righe nel dataset di training: {len(master_df)}")
print("\nDistribuzione per Schema Target:")
# Mostra quante coppie abbiamo per ogni tipo di target
print(master_df['target'].value_counts())

print(f"\nFile MASTER salvato in: {OUTPUT_MASTER}")
print("\nAnteprima finale:")
print(master_df[['source', 'target', 'anchor_id', 'target_id']].head(5))

Inizio creazione del MASTER DATASET COMPLETO...
   - Lettura: training_data_step1_cisco_ens.csv ... OK (318 righe)
   - Lettura: training_data_step2_cisco_secNumCloud.csv ... OK (567 righe)
   - Lettura: training_data_step3_cisco_bsi.csv ... OK (555 righe)
   - Lettura: training_data_step4_cisco_eucs.csv ... OK (1272 righe)
   - Lettura: training_data_step5_fabasoft_eucs.csv ... OK (206 righe)
   - Lettura: training_data_step6_fabasoft_bsi.csv ... OK (27 righe)
   - Lettura: training_data_step7_neweucs_bsi_filtered.csv ... OK (569 righe)

Mescolamento e Pulizia Finale...
   - Rimossi 6 duplicati esatti.
------------------------------
RISULTATO FINALE (MASTER DATASET)
------------------------------
Totale righe nel dataset di training: 3508

Distribuzione per Schema Target:
target
EUCS                 1270
BSI C5               1151
SecNumCloud (EN)      563
Spanish ENS (EN)      318
EUCS (via BSI C5)     206
Name: count, dtype: int64

File MASTER salvato in: TrainAndTestData/trainingDat

## Generate the 2 test data

one is the old test data from the old paper (oldEucs and medinaMetrics) and the other (the one that is made by old data but it was not use for testing) is oldEucs and bsic5

In [9]:
import pandas as pd
import json
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_OLD_EUCS = "Schemes/OldEucsRequirements.csv"
PATH_MEDINA = "Schemes/medinaMetrics.csv"
PATH_BSI = "Schemes/BSI-C5.json"

# Output folder
TEST_DIR = "TrainAndTestData/testData/"
if not os.path.exists(TEST_DIR):
    os.makedirs(TEST_DIR)

OUT_TEST_1 = os.path.join(TEST_DIR, "test_set_1_medina_oldeucs.csv")
OUT_TEST_2 = os.path.join(TEST_DIR, "test_set_2_oldeucs_bsi.csv")

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE DATI BASE
# ==============================================================================
print("1. Caricamento Schemi Base...")

# A) Old EUCS (Target per Test 1, Anchor per Test 2)
# -------------------------------------------------
try:
    df_old_eucs = pd.read_csv(PATH_OLD_EUCS)
    # Cerco colonna ID e Descrizione
    col_oe_id = next((c for c in df_old_eucs.columns if 'controlId' in c or 'ID' in c), None)
    col_oe_desc = next((c for c in df_old_eucs.columns if 'description' in c.lower()), None)
    
    if col_oe_id and col_oe_desc:
        print(f"   - Old EUCS caricato: {len(df_old_eucs)} righe.")
    else:
        print("   - ERRORE: Colonne ID/Description non trovate in Old EUCS.")
        exit()
        
    # Creo dizionario Old EUCS: ID -> Text
    oldeucs_lookup = {}
    for _, row in df_old_eucs.iterrows():
        oid = str(row[col_oe_id]).strip()
        otext = str(row[col_oe_desc]).strip()
        oldeucs_lookup[oid] = otext
        # Normalizzo ID per lookup (lowercase)
        oldeucs_lookup[oid.lower()] = otext
        
except Exception as e:
    print(f"   - ERRORE Old EUCS: {e}")
    exit()

# B) BSI C5 (Target per Test 2)
# -----------------------------
bsi_lookup = {}
try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
        
    for item in bsi_data:
        if 'code' not in item: continue
        code = str(item['code']).strip().lower()
        
        name = item.get('name', '') or ""
        desc = item.get('descriptionTitle', '') or ""
        full_text = f"{name}: {desc}".strip()
        
        bsi_lookup[code] = full_text
    print(f"   - BSI C5 caricato: {len(bsi_lookup)} controlli.")
except Exception as e:
    print(f"   - ERRORE BSI: {e}")
    exit()

# ==============================================================================
# 3. GENERAZIONE TEST SET 1: Medina Metrics -> Old EUCS
# ==============================================================================
print("\n2. Generazione Test Set 1 (Medina -> Old EUCS)...")
# Logica: In medinaMetrics.csv c'è una colonna 'controlId' che punta a Old EUCS

test1_data = []
try:
    df_medina = pd.read_csv(PATH_MEDINA)
    
    # Identifico colonne
    col_med_id = 'ID'
    col_med_desc = 'description'
    col_med_ref = 'controlId' # Punta a Old EUCS
    
    matches_1 = 0
    for idx, row in df_medina.iterrows():
        # Anchor: Metrica
        m_id = str(row[col_med_id])
        m_desc = str(row[col_med_desc])
        anchor_text = f"{m_id}: {m_desc}" # ID + Descrizione
        
        # Target: Old EUCS
        ref_id = str(row[col_med_ref]).strip()
        
        # Cerco il testo del target nel dizionario Old EUCS
        # Provo exact match o lowercase
        target_text = oldeucs_lookup.get(ref_id) or oldeucs_lookup.get(ref_id.lower())
        
        if target_text:
            test1_data.append({
                'anchor_id': m_id,
                'anchor_text': anchor_text,
                'target_id': ref_id,
                'target_text': target_text,
                'source': 'Medina Metrics',
                'target': 'Old EUCS'
            })
            matches_1 += 1
            
    # Salvataggio
    pd.DataFrame(test1_data).to_csv(OUT_TEST_1, index=False)
    print(f"   - Creato {OUT_TEST_1} con {matches_1} coppie.")

except Exception as e:
    print(f"   - ERRORE Test Set 1: {e}")

# ==============================================================================
# 4. GENERAZIONE TEST SET 2: Old EUCS -> BSI C5
# ==============================================================================
print("\n3. Generazione Test Set 2 (Old EUCS -> BSI C5)...")
# Logica: Dobbiamo capire come Old EUCS mappa su BSI.
# Se Old EUCS non ha una colonna esplicita "BSI Mapping", 
# dobbiamo inferirlo dal nome (es. OPS-05.3H -> OPS-05?) 
# O forse il mapping è implicito perché usano codici simili?
#
# ANALISI: In OldEucsRequirements.csv (dal tuo head) vedo solo ID e Description.
# Non vedo una colonna di mapping verso BSI.
# TUTTAVIA, l'ID stesso (es. "OPS-05.3H") sembra derivare dai domini BSI (OPS, IAM, etc).
#
# Se non hai un file di mapping esplicito "OldEUCS -> BSI", 
# l'unica strategia automatica è provare a matchare il prefisso del codice.
# Es: "OPS-05.3H" -> cerco "OPS-05" in BSI?
#
# Oppure, se vuoi testare se il modello capisce la semantica, possiamo creare un file
# dove il target è il controllo BSI che ha l'ID base corrispondente.

test2_data = []
matches_2 = 0

for oe_id, oe_text in oldeucs_lookup.items():
    # Provo a estrarre una radice BSI-compatibile dall'ID Old EUCS
    # Es: "OPS-05.3H" -> "ops-05"
    # Regex: Prendi lettere e numeri fino al secondo trattino o punto
    
    # Tentativo euristico: (CAT)-(NUM)
    match = re.match(r'([A-Za-z]+-\d+)', oe_id) # Cattura "OPS-05" da "OPS-05.3H"
    
    if match:
        potential_bsi_code = match.group(1).lower() # "ops-05"
        
        if potential_bsi_code in bsi_lookup:
            bsi_text = bsi_lookup[potential_bsi_code]
            
            test2_data.append({
                'anchor_id': oe_id,      # Old EUCS ID
                'anchor_text': oe_text,  # Old EUCS Text
                'target_id': potential_bsi_code, # BSI ID
                'target_text': bsi_text, # BSI Text
                'source': 'Old EUCS',
                'target': 'BSI C5 (Inferred)'
            })
            matches_2 += 1

# Salvataggio
pd.DataFrame(test2_data).to_csv(OUT_TEST_2, index=False)
print(f"   - Creato {OUT_TEST_2} con {matches_2} coppie (Match euristico su ID).")

print("-" * 30)
print("TEST SET GENERATION COMPLETED")

1. Caricamento Schemi Base...
   - Old EUCS caricato: 70 righe.
   - BSI C5 caricato: 223 controlli.

2. Generazione Test Set 1 (Medina -> Old EUCS)...
   - Creato TrainAndTestData/testData/test_set_1_medina_oldeucs.csv con 179 coppie.

3. Generazione Test Set 2 (Old EUCS -> BSI C5)...
   - Creato TrainAndTestData/testData/test_set_2_oldeucs_bsi.csv con 140 coppie (Match euristico su ID).
------------------------------
TEST SET GENERATION COMPLETED


In [10]:
import pandas as pd
import re
from difflib import SequenceMatcher

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
PATH_FABASOFT = "Schemes/fabasoftMetrics.csv"
PATH_MEDINA = "Schemes/medinaMetrics.csv"

# Soglia di similarità (0.9 = 90% simile)
SIMILARITY_THRESHOLD = 0.9 

# ==============================================================================
# 2. CARICAMENTO E PULIZIA
# ==============================================================================
print("Caricamento dataset...")

# Fabasoft
try:
    df_fab = pd.read_csv(PATH_FABASOFT, engine='python', on_bad_lines='skip')
    # Fabasoft ha la descrizione nella colonna 'Description'
    fab_descs = df_fab['Description'].dropna().astype(str).tolist()
    print(f"   - Fabasoft: {len(fab_descs)} descrizioni caricate.")
except Exception as e:
    print(f"   - Errore Fabasoft: {e}")
    fab_descs = []

# Medina
try:
    df_med = pd.read_csv(PATH_MEDINA)
    # Medina ha la descrizione nella colonna 'description'
    med_descs = df_med['description'].dropna().astype(str).tolist()
    print(f"   - Medina: {len(med_descs)} descrizioni caricate.")
except Exception as e:
    print(f"   - Errore Medina: {e}")
    med_descs = []

# ==============================================================================
# 3. ANALISI SOVRAPPOSIZIONE
# ==============================================================================
print("\nInizio confronto (Similarity Check)...")

def normalize(text):
    # Rimuove caratteri non alfanumerici e mette in lowercase
    return re.sub(r'[^a-z0-9]', '', text.lower())

# Set per lookup veloce (Exact Match)
fab_norm_set = set(normalize(d) for d in fab_descs)
med_norm_set = set(normalize(d) for d in med_descs)

# A. Exact Matches
intersection = fab_norm_set.intersection(med_norm_set)
print(f"1. Exact Matches (stessa descrizione normalizzata): {len(intersection)}")

# B. Fuzzy Matches (Controllo più lento ma accurato)
potential_leaks = []

# Per non fare un prodotto cartesiano enorme, controlliamo solo se i set sono piccoli (<1000)
if len(fab_descs) < 500 and len(med_descs) < 500:
    print("2. Fuzzy Matching in corso (potrebbe richiedere qualche secondo)...")
    for f_desc in fab_descs:
        for m_desc in med_descs:
            # Calcola similarità
            ratio = SequenceMatcher(None, f_desc, m_desc).ratio()
            if ratio >= SIMILARITY_THRESHOLD:
                potential_leaks.append((f_desc, m_desc, ratio))
else:
    print("2. Fuzzy Matching saltato (dataset troppo grandi per confronto rapido).")

# ==============================================================================
# 4. REPORT
# ==============================================================================
print("\n------------------------------------------------")
print("RISULTATO ANALISI")
print("------------------------------------------------")

if len(intersection) == 0 and len(potential_leaks) == 0:
    print("✅ NESSUNA sovrapposizione rilevata.")
    print("   Puoi usare tranquillamente Fabasoft nel Training e Medina nel Test.")
else:
    print(f"⚠️ ATTENZIONE: Rilevata possibile sovrapposizione!")
    print(f"   - Duplicati esatti: {len(intersection)}")
    print(f"   - Duplicati simili (>90%): {len(potential_leaks)}")
    
    if potential_leaks:
        print("\nEsempi di similarità trovata:")
        for f, m, r in potential_leaks[:5]:
            print(f"   Fabasoft: {f[:50]}...")
            print(f"   Medina:   {m[:50]}...")
            print(f"   Score:    {r:.2f}\n")


Caricamento dataset...
   - Fabasoft: 37 descrizioni caricate.
   - Medina: 183 descrizioni caricate.

Inizio confronto (Similarity Check)...
1. Exact Matches (stessa descrizione normalizzata): 0
2. Fuzzy Matching in corso (potrebbe richiedere qualche secondo)...

------------------------------------------------
RISULTATO ANALISI
------------------------------------------------
✅ NESSUNA sovrapposizione rilevata.
   Puoi usare tranquillamente Fabasoft nel Training e Medina nel Test.


# Data visualization

In [11]:
import pandas as pd

train = pd.read_csv("TrainAndTestData/trainingData/MASTER_TRAINING_DATA.csv")
test1 = pd.read_csv("TrainAndTestData/testData/test_set_1_medina_oldeucs.csv")
test2 = pd.read_csv("TrainAndTestData/testData/test_set_2_oldeucs_bsi.csv") 

train

,anchor_id,anchor_text,target_id,target_text,source,target,original_file
0,CCF 286,Supplier Management Program: A supplier manage...,pm-02.3,Following the risk assessment of a subservice ...,Cisco,EUCS,training_data_step4_cisco_eucs.csv
1,CCF 59,Device and Hardware Transfer: Devices and hard...,mp.si.2,Cryptography,Cisco,Spanish ENS (EN),training_data_step1_cisco_ens.csv
2,CCF 28,SDLC Methodology: A formal [The Organization] ...,6.5.a,The service provider must document an estimate...,Cisco,SecNumCloud (EN),training_data_step2_cisco_secNumCloud.csv
3,CCF 8,Cybersecurity Legal and Regulatory Requirement...,18.1.a,"The service provider must identify the legal, ...",Cisco,SecNumCloud (EN),training_data_step2_cisco_secNumCloud.csv
4,CCF 175,Password Management & Configuration: Passwords...,iam-07.2,The access to all environments of the CSP shal...,Cisco,EUCS,training_data_step4_cisco_eucs.csv
...,...,...,...,...,...,...,...
3503,CCF 133,Policies and Standards over Metadata: Policies...,ops-12,LOGGING AND MONITORING – IDENTIFICATION OF EVE...,Cisco,BSI C5,training_data_step3_cisco_bsi.csv
3504,CCF 246,Deletion of PII: [The Organization] either del...,pi-03,SECURE DELETION OF DATA: CSC data is securely ...,Cisco,BSI C5,training_data_step3_cisco_bsi.csv
3505,CCF 315,Infrastructure & Application patching: Infrast...,12.11.a,The service provider must document and impleme...,Cisco,SecNumCloud (EN),training_data_step2_cisco_secNumCloud.csv
3506,PSS-03.2,The CSP shall validate the functionality of th...,pss-10,Software Defined Networking: If the Cloud Serv...,NewEUCS (Filtered),BSI C5,training_data_step7_neweucs_bsi_filtered.csv


In [12]:
test1

,anchor_id,anchor_text,target_id,target_text,source,target
0,1,1: This metric is used to assess if the antima...,OPS-05.3H,The CSP shall automatically monitor the system...,Medina Metrics,Old EUCS
1,2,2: This metric is used to assess if the antima...,OPS-05.3H,The CSP shall automatically monitor the system...,Medina Metrics,Old EUCS
2,3,3: This metric is used to assess if backups ar...,OPS-07.2H,In order to check the proper application of th...,Medina Metrics,Old EUCS
3,3,3: This metric is used to assess if backups ar...,OPS-09.2H,When the backup data is transmitted to a remot...,Medina Metrics,Old EUCS
4,4,4: This metric is used to assess the configure...,OPS-07.2H,In order to check the proper application of th...,Medina Metrics,Old EUCS
...,...,...,...,...,...,...
174,163,163: Where is access control monitored and reg...,PS-02.8H,The access control policy shall include loggin...,Medina Metrics,Old EUCS
175,164,164: Which tool is used for monitoring the lis...,ISP-03.5H,The list of exceptions shall be automatically ...,Medina Metrics,Old EUCS
176,165,165: Which processes are in place for event de...,OPS-12.2H,The CSP shall automatically monitor that event...,Medina Metrics,Old EUCS
177,166,166: How are changes in role and right managem...,CCM-05.1H,The CSP shall define roles and rights accordin...,Medina Metrics,Old EUCS


In [13]:
test2

,anchor_id,anchor_text,target_id,target_text,source,target
0,OPS-05.3H,The CSP shall automatically monitor the system...,ops-05,PROTECTION AGAINST MALWARE – IMPLEMENTATION: M...,Old EUCS,BSI C5 (Inferred)
1,ops-05.3h,The CSP shall automatically monitor the system...,ops-05,PROTECTION AGAINST MALWARE – IMPLEMENTATION: M...,Old EUCS,BSI C5 (Inferred)
2,OPS-07.2H,In order to check the proper application of th...,ops-07,DATA BACKUP AND RECOVERY – MONITORING: The pro...,Old EUCS,BSI C5 (Inferred)
3,ops-07.2h,In order to check the proper application of th...,ops-07,DATA BACKUP AND RECOVERY – MONITORING: The pro...,Old EUCS,BSI C5 (Inferred)
4,OPS-13.1H,The CSP shall store all log data in an integri...,ops-13,"LOGGING AND MONITORING – ACCESS, STORAGE AND D...",Old EUCS,BSI C5 (Inferred)
...,...,...,...,...,...,...
135,ops-12.2h,The CSP shall automatically monitor that event...,ops-12,LOGGING AND MONITORING – IDENTIFICATION OF EVE...,Old EUCS,BSI C5 (Inferred)
136,CCM-05.1H,The CSP shall define roles and rights accordin...,ccm-05,PERFORMING AND LOGGING CHANGES: Changes to the...,Old EUCS,BSI C5 (Inferred)
137,ccm-05.1h,The CSP shall define roles and rights accordin...,ccm-05,PERFORMING AND LOGGING CHANGES: Changes to the...,Old EUCS,BSI C5 (Inferred)
138,ISP-02.4H,The CSP’s subject matter experts shall review ...,isp-02,SECURITY POLICIES AND PROCEDURES: Policies and...,Old EUCS,BSI C5 (Inferred)


In [14]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

# 1. CONFIGURAZIONE BACKEND LATEX
# Nota: Questo backend genera file PDF di alta qualità ma non mostra finestre a schermo.
matplotlib.use("pgf")
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": [],
    "axes.labelsize": 16,
    "font.size": 14,
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "pgf.rcfonts": False,
})

# 2. PREPARAZIONE DATI
# Assumiamo che 'train' sia già caricato nel notebook
if 'train' not in locals():
    print("ERRORE: Il DataFrame 'train' non è definito.")
else:
    # Creiamo la tabella incrociata
    df_plot = pd.crosstab(train['target'], train['source'])
    # Ordiniamo per totale decrescente
    df_plot = df_plot.loc[df_plot.sum(axis=1).sort_values(ascending=False).index]

    # 3. DEFINIZIONE COLORI RICHIESTI
    custom_colors = ['tab:blue', 'tab:orange', 'tab:green', 'gold']

    # 4. CREAZIONE GRAFICO (Oggetto Figure)
    fig, ax = plt.subplots(figsize=(10, 6))

    df_plot.plot(
        kind='bar', 
        stacked=True, 
        color=custom_colors, 
        edgecolor='black', 
        linewidth=0.8,
        width=0.6,
        ax=ax
    )

    # 5. AGGIUNTA PATTERN
    patterns = ['//', '\\\\', 'xx', '..']
    for i, container in enumerate(ax.containers):
        hatch = patterns[i % len(patterns)]
        for patch in container:
            patch.set_hatch(hatch)

    # 6. ETICHETTE TOTALI
    totals = df_plot.sum(axis=1)
    for i, total in enumerate(totals):
        ax.text(
            i, 
            total + (total * 0.02), 
            int(total), 
            ha='center', 
            va='bottom', 
            fontsize=12, 
            color='black'
        )

    # 7. FORMATTAZIONE
    ax.set_title("") 
    ax.set_xlabel("Target Standard (Positive)", fontsize=16, labelpad=10)
    ax.set_ylabel("Number of Semantic Pairs", fontsize=16, labelpad=10)
    plt.xticks(rotation=0)

    ax.legend(
        title="Source (Anchor)", 
        title_fontsize=12, 
        loc='upper right', 
        frameon=True, 
        framealpha=0.95
    )

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # 8. SALVATAGGIO PDF
    output_path = 'PaperPlots/train_composition_latex.pdf'
    # Assicuriamoci che la cartella esista (opzionale, ma consigliato)
    import os
    os.makedirs('PaperPlots', exist_ok=True)
    
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight')
    plt.close(fig) # Chiude la figura per liberare memoria

    # ==============================================================================
    # 9. OUTPUT TESTUALE NEL NOTEBOOK (DATA SUMMARY)
    # ==============================================================================
    print("-" * 60)
    print("GRAFICO GENERATO CORRETTAMENTE")
    print("-" * 60)
    print(f"File salvato in: {output_path}")
    print("\nRIEPILOGO DATI PLOTTATI (Textual View):")
    print("-" * 60)
    
    # Calcoliamo i totali per colonna per visualizzarli
    df_summary = df_plot.copy()
    df_summary['TOTAL'] = df_summary.sum(axis=1)
    
    # Formattiamo l'output come una tabella Markdown-like
    header = f"{'TARGET STANDARD':<30} | {'SOURCE BREAKDOWN':<40} | {'TOTAL':>6}"
    print(header)
    print("-" * len(header))
    
    for target_name, row in df_summary.iterrows():
        total = row['TOTAL']
        # Costruiamo una stringa che dice es: "Cisco (300), Fabasoft (50)"
        breakdown_parts = []
        for source_col in df_plot.columns:
            val = row[source_col]
            if val > 0:
                breakdown_parts.append(f"{source_col}: {val}")
        
        breakdown_str = ", ".join(breakdown_parts)
        
        # Stampa riga formattata
        print(f"{target_name:<30} | {breakdown_str:<40} | {total:>6}")
        
    print("-" * 60)
    print("Nota: Il grafico PDF vettoriale include pattern e font LaTeX.")

------------------------------------------------------------
GRAFICO GENERATO CORRETTAMENTE
------------------------------------------------------------
File salvato in: PaperPlots/train_composition_latex.pdf

RIEPILOGO DATI PLOTTATI (Textual View):
------------------------------------------------------------
TARGET STANDARD                | SOURCE BREAKDOWN                         |  TOTAL
----------------------------------------------------------------------------------
EUCS                           | Cisco: 1270                              |   1270
BSI C5                         | Cisco: 555, Fabasoft Metrics: 27, NewEUCS (Filtered): 569 |   1151
SecNumCloud (EN)               | Cisco: 563                               |    563
Spanish ENS (EN)               | Cisco: 318                               |    318
EUCS (via BSI C5)              | Fabasoft Metrics: 206                    |    206
------------------------------------------------------------
Nota: Il grafico PDF vettorial

In [15]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 1. CONFIGURAZIONE LATEX
matplotlib.use("pgf")
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": [],
    "axes.labelsize": 16,
    "font.size": 14,
    "legend.fontsize": 12,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "pgf.rcfonts": False,
})

# 2. DATI
# Test Set A: Medina (Source) -> Old EUCS (Target) -> 179 items
# Test Set B: Old EUCS (Source) -> BSI C5 (Target) -> 140 items
sets = ['Test Set A\n(Benchmark)', 'Test Set B\n(Generalization)']
counts = [179, 140]

# 3. DEFINIZIONE STILE VISIVO (La "Grammatica" del grafico)
# Colore = SORGENTE
color_map = {
    'Medina Metrics': 'tab:purple',
    'Old EUCS': 'tab:blue'
}
# Pattern = TARGET
hatch_map = {
    'Old EUCS': '///',   # Righe diagonali
    'BSI C5': '...'      # Puntini
}

# Associazione logica per ogni barra
bar_configs = [
    {'source': 'Medina Metrics', 'target': 'Old EUCS', 'count': 179},
    {'source': 'Old EUCS',       'target': 'BSI C5',   'count': 140}
]

# 4. CREAZIONE GRAFICO
fig, ax = plt.subplots(figsize=(9, 6))

# Disegniamo le barre una per una per controllarne colore e pattern
bars = []
for i, config in enumerate(bar_configs):
    color = color_map[config['source']]
    hatch = hatch_map[config['target']]
    
    # Disegna la barra
    bar = ax.bar(
        i, 
        config['count'], 
        color=color, 
        edgecolor='black', 
        hatch=hatch,     # <--- Qui sta la magia visiva
        width=0.5,
        linewidth=1.0,
        alpha=0.9
    )
    
    # Etichetta col numero sopra
    ax.text(
        i, 
        config['count'] + 5, 
        str(config['count']), 
        ha='center', 
        va='bottom', 
        fontweight='bold', 
        fontsize=14
    )

# 5. CREAZIONE LEGENDA SEMANTICA (Source vs Target)
# Creiamo due gruppi di legende per spiegare la visual grammar

# Gruppo 1: Sorgenti (Colori)
legend_handles = []
legend_handles.append(mpatches.Patch(color='none', label=r"\textbf{SOURCE DATA (Color):}")) # Titolo fittizio
legend_handles.append(mpatches.Patch(facecolor='tab:purple', edgecolor='black', label='Medina Metrics'))
legend_handles.append(mpatches.Patch(facecolor='tab:blue', edgecolor='black', label='Old EUCS Reqs'))

# Spazio vuoto
legend_handles.append(mpatches.Patch(color='none', label='')) 

# Gruppo 2: Target (Pattern)
legend_handles.append(mpatches.Patch(color='none', label=r"\textbf{TARGET STANDARD (Pattern):}")) # Titolo fittizio
# Per mostrare solo il pattern nella legenda usiamo facecolor='white' e hatch nero
legend_handles.append(mpatches.Patch(facecolor='white', edgecolor='black', hatch='///', label='Old EUCS'))
legend_handles.append(mpatches.Patch(facecolor='white', edgecolor='black', hatch='...', label='BSI C5'))

# Posizionamento Legenda
ax.legend(
    handles=legend_handles, 
    loc='upper right', 
    bbox_to_anchor=(1.45, 1.02), # Fuori dal grafico a destra
    frameon=True,
    fontsize=11
)

# 6. FORMATTAZIONE ASSI
ax.set_xticks([0, 1])
ax.set_xticklabels(sets)
ax.set_ylabel("Number of Evaluation Pairs", fontsize=16, labelpad=10)
ax.set_xlabel("Evaluation Scenario", fontsize=16, labelpad=10)
ax.set_ylim(0, 200) # Un po' di aria sopra

# Rimuovi bordi inutili
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# 7. SALVATAGGIO
plt.tight_layout()
output_path = 'PaperPlots/test_composition_visual_latex.pdf'
plt.savefig(output_path, bbox_inches='tight')

# Output testuale per il notebook
print("-" * 60)
print("GRAFICO GENERATO: Visualizzazione Semantica dei Test Set")
print("-" * 60)
print(f"File salvato in: {output_path}")
print("\nLegenda Visuale Applicata:")
print("1. Test Set A (Benchmark):")
print("   - COLORE VIOLA (Sorgente)  = Medina Metrics")
print("   - PATTERN RIGHE (Target)   = Old EUCS")
print("\n2. Test Set B (Generalization):")
print("   - COLORE BLU (Sorgente)    = Old EUCS Reqs")
print("   - PATTERN PUNTINI (Target) = BSI C5")
print("-" * 60)

------------------------------------------------------------
GRAFICO GENERATO: Visualizzazione Semantica dei Test Set
------------------------------------------------------------
File salvato in: PaperPlots/test_composition_visual_latex.pdf

Legenda Visuale Applicata:
1. Test Set A (Benchmark):
   - COLORE VIOLA (Sorgente)  = Medina Metrics
   - PATTERN RIGHE (Target)   = Old EUCS

2. Test Set B (Generalization):
   - COLORE BLU (Sorgente)    = Old EUCS Reqs
   - PATTERN PUNTINI (Target) = BSI C5
------------------------------------------------------------
